In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Đọc dữ liệu
df = pd.read_csv("/kaggle/input/aio-2025-linear-forecasting-challenge/FPT_train.csv")

# Đảm bảo đã sort theo thời gian nếu cần (nếu file đã sort sẵn thì bỏ qua cũng được)
if "time" in df.columns:
    df = df.sort_values("time").reset_index(drop=True)

# Lưu lại giá close cuối cùng để dự báo 100 ngày tiếp theo
last_close = df["close"].iloc[-1]

# 2. Tạo dữ liệu supervised: dùng close[t] dự báo vector [close[t+1], ..., close[t+100]]
max_horizon = 100

for h in range(1, max_horizon + 1):
    df[f"close_t+{h}"] = df["close"].shift(-h)

# Loại bỏ các dòng không đủ 100 ngày tương lai
df_supervised = df.dropna().reset_index(drop=True)

# Input: close[t]
X = df_supervised[["close"]].values

# Output: 100 điểm tương lai
y_cols = [f"close_t+{h}" for h in range(1, max_horizon + 1)]
y = df_supervised[y_cols].values  # shape: [n_samples, 100]

# 3. Tách train test (80 20) để tính MSE
split = int(len(df_supervised) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Tính MSE trung bình trên toàn bộ 100 horizon
mse = mean_squared_error(y_test.ravel(), y_pred.ravel())
print("MSE (train 80%, test 20%, multi output 100 ngày):", mse)

# 4. Train lại trên FULL dữ liệu trước khi dự báo 100 ngày tiếp theo
model_full = LinearRegression()
model_full.fit(X, y)

print("Model full data - Intercept shape:", model_full.intercept_.shape)
print("Model full data - Coef shape:", model_full.coef_.shape)

# 5. Dự báo 100 ngày tiếp theo từ giá close cuối cùng trong dữ liệu
#    Output một lần là vector 100 điểm
future_100 = model_full.predict([[last_close]])[0]  # shape (100,)

# 6. Tạo dataframe submission: id 1..100, close = dự báo
submission = pd.DataFrame({
    "id": np.arange(1, max_horizon + 1),
    "close": future_100
})

submission.to_csv("submission.csv", index=False)

print("Saved submission.csv, preview:")
print(submission.head())


MSE (train 80%, test 20%, multi output 100 ngày): 247.0130475098989
Model full data - Intercept shape: (100,)
Model full data - Coef shape: (100, 1)
Saved submission.csv, preview:
   id       close
0   1  120.278683
1   2  120.433793
2   3  120.593833
3   4  120.763075
4   5  120.936020


/tmp/ipykernel_47/3451720613.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"close_t+{h}"] = df["close"].shift(-h)
/tmp/ipykernel_47/3451720613.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"close_t+{h}"] = df["close"].shift(-h)
/tmp/ipykernel_47/3451720613.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, u